<a href="https://colab.research.google.com/github/HanByeongGyu/self-driving-car_project/blob/main/imagenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/data

/content/drive/.shortcut-targets-by-id/1SnYc60S9PxgI5cT34bhrD4sZWGMkoT5x/data


In [ ]:
%ls

ILSVRC2012_devkit_t12.tar.gz  ILSVRC2012_img_val.tar
ILSVRC2012_devkit_t3.tar.gz   meta.bin


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import torchvision.models as models
import torch.nn as nn

In [ ]:
torch.cuda.is_available()

True

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import torchvision.models as models
import torch.nn as nn

def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, largest = True, sorted = True)
        pred = pred.t()
        correct = (pred == target.unsqueeze(dim=0)).expand_as(pred)

        res = []
        for k in topk:

            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(1.0 / batch_size))
        return res


val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

valdate = torchvision.datasets.ImageNet('/content/drive/MyDrive/data', split='val',download=None, transform=val_transform)
valLoader = torch.utils.data.DataLoader(valdate, batch_size=4, shuffle=True)



alexnet = models.alexnet(pretrained=True)
alexnet.cuda()
alexnet.eval()
cri = nn.CrossEntropyLoss().cuda()
c = 0
sum = 0;
average = 0;
for iterlate, (ip, t) in enumerate(valLoader):
    ip = ip.cuda()
    t = t.cuda()
    op = alexnet(ip)
    l = cri(op, t)
    c+=1
    prec = accuracy(op.data, t, topk=(1,5))
    value = prec[0].item()
    n = ip.size(0)
    sum+=value
    average = sum/c;
    print("진행율 {}/{}, 정확도:({:.2f}), 평균:({:.3f})".format(c,len(valLoader), value, average))
print("alexnet의 Top1 정확도: ", average*100)
alexnet_average = average*100

In [ ]:
round(alexnet_average,3)

56.522

In [ ]:
ResNet18 = models.resnet18(pretrained = True)
ResNet18.cuda()
ResNet18.eval()
cri = nn.CrossEntropyLoss().cuda()
c = 0
sum = 0;
average = 0;
for iterlate, (ip, t) in enumerate(valLoader):
    ip = ip.cuda()
    t = t.cuda()
    op = ResNet18(ip)
    l = cri(op, t)
    c+=1
    prec = accuracy(op.data, t, topk=(1,5))
    value = prec[0].item()
    n = ip.size(0)
    sum+=value
    average = sum/c;
    print("진행율 {}/{}, 정확도:({:.2f}), 평균:({:.3f})".format(c,len(valLoader), value, average))
print("ResNet18의 Top1 정확도: ", average*100)
ResNet18_average = average*100

In [ ]:
VGG16 = models.vgg16(pretrained = True)
VGG16.cuda()
VGG16.eval()

cri = nn.CrossEntropyLoss().cuda()
c = 0
sum = 0;
average = 0;
for iterlate, (ip, t) in enumerate(valLoader):
    ip = ip.cuda()
    t = t.cuda()
    op = VGG16(ip)
    l = cri(op, t)
    c+=1
    prec = accuracy(op.data, t, topk=(1,5))
    value = prec[0].item()
    n = ip.size(0)
    sum+=value
    average = sum/c;
    print("진행율 {}/{}, 정확도:({:.2f}), 평균:({:.3f})".format(c,len(valLoader), value, average))
print("VGG16의 Top1 정확도: ", average*100)
VGG16_average = average*100

In [ ]:
GoogLeNet = models.googlenet(pretrained=True)
GoogLeNet.cuda()
GoogLeNet.eval

cri = nn.CrossEntropyLoss().cuda()
c = 0
sum = 0;
average = 0;
for iterlate, (ip, t) in enumerate(valLoader):
    ip = ip.cuda()
    t = t.cuda()
    op = GoogLeNet(ip)
    l = cri(op, t)
    c+=1
    prec = accuracy(op.data, t, topk=(1,5))
    value = prec[0].item()
    n = ip.size(0)
    sum+=value
    average = sum/c;
    print("진행율 {}/{}, 정확도:({:.2f}), 평균:({:.3f})".format(c,len(valLoader), value, average))
print("GoogLeNet의 Top1 정확도: ", average*100)
GoogLeNet_average = average*100

In [ ]:
GoogLeNet_average

48.954

In [ ]:
import pandas as pd

In [ ]:
Output = pd.DataFrame({'Model' : ['alexnet', 'ResNet18', 'VGG16', 'GoogLeNet'], 'Average' : [alexnet_average, ResNet18_average, VGG16_average, GoogLeNet_average]}).sort_values(by = 'Average', ascending = False).reset_index()
Output

,index,Model,Average
0,2,VGG16,71.592
1,1,ResNet18,69.758
2,0,alexnet,56.522
3,3,GoogLeNet,48.954


In [ ]:
Output.drop('index', axis = 1)

,Model,Average
0,VGG16,71.592
1,ResNet18,69.758
2,alexnet,56.522
3,GoogLeNet,48.954
